In [1]:
import sys

sys.path.append("../")

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import entropy, kurtosis
from config.transformers import *
from config.models import *
from config.loss_functions import *
from config.validation import *
from config.regressors import *
import numpy as np
import pyriemann
import pywt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'
DATASET = 'freemoves' # change this to guided/freemoves if needed

X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

tw_extractor = TimeWindowTransformer(size = 500, step = 50)
label_extractor = LabelWindowExtractor(size = 500, step = 50)

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)

In [4]:
rmse_scores = cross_validate_dann(X_windows, Y_labels, tensor_dataset=DANNWindowDataset)


=== Fold 1 | Train on [1, 2, 3, 4], Validate on 0 ===
Epoch 01 | Train Loss: 656.3550 | Val RMSE: 24.1060 | Dom Train Acc: 38.71%
           λ = 0.100
Epoch 02 | Train Loss: 601.7190 | Val RMSE: 23.0802 | Dom Train Acc: 41.36%
           λ = 0.200
Epoch 03 | Train Loss: 535.3066 | Val RMSE: 21.8700 | Dom Train Acc: 47.94%
           λ = 0.300
Epoch 04 | Train Loss: 469.7959 | Val RMSE: 19.8016 | Dom Train Acc: 48.97%
           λ = 0.400
Epoch 05 | Train Loss: 405.1439 | Val RMSE: 19.7694 | Dom Train Acc: 50.60%
           λ = 0.500
Epoch 06 | Train Loss: 344.8084 | Val RMSE: 18.4050 | Dom Train Acc: 50.66%
           λ = 0.600
Epoch 07 | Train Loss: 291.3523 | Val RMSE: 18.3262 | Dom Train Acc: 52.48%
           λ = 0.700
Epoch 08 | Train Loss: 247.0739 | Val RMSE: 15.8564 | Dom Train Acc: 53.44%
           λ = 0.800
Epoch 09 | Train Loss: 208.1713 | Val RMSE: 14.9946 | Dom Train Acc: 52.37%
           λ = 0.900
Epoch 10 | Train Loss: 175.3392 | Val RMSE: 13.0051 | Dom Train Acc: 56.

In [3]:
def RMSE(y_pred, y_val):
    return (np.sum((y_val - y_pred) ** 2) / np.prod(y_val.shape)) ** 0.5

def NMSE(y_pred, y_val):
    num = np.sum((y_val - y_pred) ** 2)
    den = np.sum((y_val - np.mean(y_val, axis=0)) ** 2)
    return num / den

class DANNTrainer2:
    def __init__(
        self,
        model: nn.Module,
        X: np.ndarray,
        Y: np.ndarray,
        train_sessions: list,
        val_session: int,
        lambda_grl: float = 0.1,
        gamma_entropy=0.5,
        batch_size: int = 512,
        max_epochs: int = 50,
        patience: int = 10,
        learning_rate: float = 1e-3,
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
        tensor_dataset=None
    ):
        self.device = device
        self.model = model.to(self.device)
        self.lambda_grl = lambda_grl
        self.gamma_entropy = gamma_entropy
        self.max_epochs = max_epochs
        self.patience = patience
        self.batch_size = batch_size
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.reg_loss = nn.MSELoss()
        self.dom_loss = nn.CrossEntropyLoss()
        self.train_loader = None  # to be assigned externally

    def train(self, validate=True):
        best_val_rmse = float("inf")
        best_weights = None
        patience_counter = 0

        def compute_lambda(epoch, max_lambda=1.0, warmup_epochs=10):
            return min(max_lambda, epoch / warmup_epochs)

        for epoch in range(1, self.max_epochs + 1):
            self.model.train()
            train_y_true, train_y_pred = [], []
            epoch_losses = []

            lambda_grl = compute_lambda(epoch, max_lambda=1.0, warmup_epochs=10)

            for x, y, sid in self.train_loader:
                x, y, sid = x.to(self.device), y.to(self.device), sid.to(self.device)
                y_pred, dom_pred = self.model(x, lambda_grl=lambda_grl)

                loss = (
                    self.reg_loss(y_pred, y) +
                    lambda_grl * self.dom_loss(dom_pred, sid)
                )
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_losses.append(loss.item())

                train_y_true.append(y.detach().cpu().numpy())
                train_y_pred.append(y_pred.detach().cpu().numpy())

            train_y_true = np.vstack(train_y_true)
            train_y_pred = np.vstack(train_y_pred)
            train_rmse = RMSE(train_y_pred, train_y_true)
            train_nmse = NMSE(train_y_pred, train_y_true)

            if validate:
                val_rmse = self.evaluate()
                domain_train_acc = self.evaluate_domain_on_train()
                print(f"Epoch {epoch:02d} | Train RMSE: {train_rmse:.4f} | Train NMSE: {train_nmse:.4f} | Val RMSE: {val_rmse:.4f} | Dom Train Acc: {domain_train_acc:.2%}")
                print(f"           λ = {lambda_grl:.3f}")

                if val_rmse < best_val_rmse:
                    best_val_rmse = val_rmse
                    best_weights = self.model.state_dict()
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= self.patience:
                        print("Early stopping triggered.")
                        break
            else:
                print(f"Epoch {epoch:02d} | Train RMSE: {train_rmse:.4f} | Train NMSE: {train_nmse:.4f}")
                print(f"           λ = {lambda_grl:.3f}")

        if validate and best_weights is not None:
            self.model.load_state_dict(best_weights)
        return best_val_rmse if validate else self.model

    def predict(self, data_loader):
        self.model.eval()
        all_preds = []
        with torch.no_grad():
            for x, *_ in data_loader:
                x = x.to(self.device)
                preds = self.model(x)[0]  # get y_pred only
                all_preds.append(preds.cpu().numpy())
        return np.vstack(all_preds)


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from config.models import DANNTrainer

# --- Load full training set ---
PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'
DATASET = 'freemoves'

X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

tw_extractor = TimeWindowTransformer(size=500, step=50)
label_extractor = LabelWindowExtractor(size=500, step=50)

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)

# X_all = X_windows.reshape(-1, 8, 500)
# Y_all = Y_labels.reshape(-1, 51)

session_ids = np.concatenate([
    np.full(X_windows[i].shape[0], i) for i in range(len(X_windows))
])

class DANNWindowDataset2(Dataset):
    def __init__(self, X, Y, session_ids):
        self.X = X.reshape(-1, *X.shape[2:])  # (N, 8, 500)
        self.Y = Y.reshape(-1, Y.shape[-1])   # (N, 51)
        self.session_ids = torch.tensor(session_ids, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.Y[idx], dtype=torch.float32),
            self.session_ids[idx],
        )


train_dataset = DANNWindowDataset2(X_windows, Y_labels, session_ids)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

dummy_X = np.empty((1, 1, 8, 500), dtype=np.float32)
dummy_Y = np.empty((1, 1, 51), dtype=np.float32)
dummy_ids = np.zeros((1, 1), dtype=int)


# --- Define model ---
model = DANNModel(lambda_grl=1.0, num_domains=5, output_dim=51)

trainer = DANNTrainer2(
    model=model,
    X=dummy_X,
    Y=dummy_Y,
    train_sessions=[0],
    val_session=0,
    lambda_grl=1.0,
    gamma_entropy=0.0,
    batch_size=512,
    max_epochs=500,
    patience=10,
    learning_rate=1e-3,
    device='cuda',
    tensor_dataset=DANNWindowDataset2
)


# Inject trainer internals
trainer.train_loader = train_loader
trainer.val_loader = None
trainer.reg_loss = torch.nn.MSELoss()
trainer.dom_loss = torch.nn.CrossEntropyLoss()

# --- Train on full data ---
trainer.train(validate=False)

# --- Load test set ---
X_test = np.load(PATH + f"{DATASET}/{DATASET}_testset_X.npy")
X_test_windows = tw_extractor.transform(X_test).reshape(-1, 8, 500)
test_loader = DataLoader(
    TensorDataset(torch.tensor(X_test_windows, dtype=torch.float32)),
    batch_size=128, shuffle=False
)

# --- Predict and save ---
predictions = trainer.predict(test_loader)
np.save(PATH + f"{DATASET}_testset_predictions.npy", predictions)


Epoch 01 | Train RMSE: 23.2175 | Train NMSE: 3.3718
           λ = 0.100
Epoch 02 | Train RMSE: 16.1987 | Train NMSE: 1.6413
           λ = 0.200
Epoch 03 | Train RMSE: 11.5368 | Train NMSE: 0.8325
           λ = 0.300
Epoch 04 | Train RMSE: 9.8828 | Train NMSE: 0.6109
           λ = 0.400
Epoch 05 | Train RMSE: 9.3169 | Train NMSE: 0.5430
           λ = 0.500
Epoch 06 | Train RMSE: 9.0003 | Train NMSE: 0.5067
           λ = 0.600
Epoch 07 | Train RMSE: 8.8032 | Train NMSE: 0.4847
           λ = 0.700
Epoch 08 | Train RMSE: 8.6630 | Train NMSE: 0.4694
           λ = 0.800
Epoch 09 | Train RMSE: 8.5286 | Train NMSE: 0.4550
           λ = 0.900
Epoch 10 | Train RMSE: 8.3746 | Train NMSE: 0.4387
           λ = 1.000
Epoch 11 | Train RMSE: 8.2355 | Train NMSE: 0.4242
           λ = 1.000
Epoch 12 | Train RMSE: 8.1317 | Train NMSE: 0.4136
           λ = 1.000
Epoch 13 | Train RMSE: 8.0285 | Train NMSE: 0.4032
           λ = 1.000
Epoch 14 | Train RMSE: 7.9746 | Train NMSE: 0.3978
          